# Data Loading and Setup

First, we'll set up our environment and load the data once, making it available to both Phase 1 and Phase 2.

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from matplotlib.patches import FancyArrowPatch
import random
import io
import pandas as pd
from PIL import Image, ImageDraw
import os
import time
from tqdm import tqdm
import uuid
import warnings
import traceback
import json
from scipy.spatial.distance import cosine
import matplotlib
import requests
from io import StringIO
import gdown
matplotlib.use('Agg')  # Use non-interactive backend for better memory handling

# Create necessary directories
output_dir = '../Data/Output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load ConceptNet data from Google Drive links
print("Loading ConceptNet data from Google Drive...")

# Define the Google Drive links
english_dataset_id = "1zBwy1rhJh-7ESQWVENkz1z1_G2aleu7-"  # ID from the Google Drive link
german_dataset_id = "10Rb0sn4uZVUJSufp08t1wzrLL1jAnRHH"  # ID from the Google Drive link

# Define file paths for caching
english_cache_path = '../Data/Input/conceptnet-assertions-5.7.0.en.tsv'
german_cache_path = '../Data/Input/conceptnet-assertions-5.7.0.de.tsv'

# Create the Input directory if it doesn't exist
input_dir = '../Data/Input'
if not os.path.exists(input_dir):
    os.makedirs(input_dir)

# Function to download data
def download_from_gdrive(file_id, output_path):
    if os.path.exists(output_path):
        print(f"Using cached data at {output_path}")
        return True
    
    print(f"Downloading data to {output_path}...")
    try:
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
        return True
    except Exception as e:
        print(f"Error downloading file: {e}")
        return False

# Download and load the datasets
english_conceptnet = None
german_conceptnet = None

# Try to download English dataset
if download_from_gdrive(english_dataset_id, english_cache_path):
    english_conceptnet = pd.read_csv(
        english_cache_path,
        sep='\t',
        names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText']
    )
    print(f"English ConceptNet loaded with {len(english_conceptnet)} assertions.")
else:
    print("Failed to load English dataset. Will use a smaller test dataset if available.")

# Try to download German dataset
if download_from_gdrive(german_dataset_id, german_cache_path):
    german_conceptnet = pd.read_csv(
        german_cache_path,
        sep='\t',
        names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText']
    )
    print(f"German ConceptNet loaded with {len(german_conceptnet)} assertions.")
else:
    print("Failed to load German dataset. Will use a smaller test dataset if available.")

# If downloading fails, try to use any existing files
if english_conceptnet is None and os.path.exists(english_cache_path):
    try:
        english_conceptnet = pd.read_csv(
            english_cache_path,
            sep='\t',
            names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText']
        )
        print(f"Using existing English ConceptNet with {len(english_conceptnet)} assertions.")
    except Exception as e:
        print(f"Error loading existing English data: {e}")

if german_conceptnet is None and os.path.exists(german_cache_path):
    try:
        german_conceptnet = pd.read_csv(
            german_cache_path,
            sep='\t',
            names=['URI', 'rel', 'start', 'end', 'weight', 'source', 'id', 'dataset', 'surfaceText']
        )
        print(f"Using existing German ConceptNet with {len(german_conceptnet)} assertions.")
    except Exception as e:
        print(f"Error loading existing German data: {e}")

# Set a flag to track if we need to run Phase 1 data processing
data_loaded = english_conceptnet is not None or german_conceptnet is not None
print(f"Data loading complete. Datasets available: {data_loaded}")

Loading ConceptNet data from Google Drive...
Using cached data at ../Data/Input/conceptnet-assertions-5.7.0.en.tsv
English ConceptNet loaded with 3423004 assertions.
Using cached data at ../Data/Input/conceptnet-assertions-5.7.0.de.tsv
German ConceptNet loaded with 1078946 assertions.
Data loading complete. Datasets available: True


# Semantica relation GRaph visualized

# Phase 1

In [17]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from matplotlib.patches import FancyArrowPatch
import random
import io
import pandas as pd
from PIL import Image, ImageDraw
import os
import time
from tqdm import tqdm
import uuid
import warnings
import traceback
import json  # Added missing import
from scipy.spatial.distance import cosine  # Added missing import
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for better memory handling

In [18]:

class ConceptNetProcessor:
    """
    Process ConceptNet data for semantic visualization
    """
    def __init__(self, english_data=None, german_data=None):
        self.english_data = english_data
        self.german_data = german_data
        self.semantic_graph = nx.DiGraph()
        self.concept_vectors = {}
        self.relation_types = set()
        
        print("ConceptNetProcessor initialized")
    
    def clean_concept_name(self, concept_str):
        """Extract clean concept name from ConceptNet format"""
        if not isinstance(concept_str, str):
            return "unknown"
            
        # Extract the concept name from the ConceptNet URI format
        parts = concept_str.split('/')
        if len(parts) >= 4:
            # Format is typically /c/LANG/CONCEPT
            concept = parts[-1]
            # Remove part-of-speech tags if present
            if '/' in concept:
                concept = concept.split('/')[0]
            return concept
        return concept_str
    
    def extract_relation_type(self, relation_str):
        """Extract relation type from ConceptNet format"""
        if not isinstance(relation_str, str):
            return "unknown"
            
        parts = relation_str.split('/')
        if len(parts) >= 3:
            # Format is typically /r/RELATION_TYPE
            return parts[-1]
        return relation_str
    
    def extract_language(self, concept_str):
        """Extract language from ConceptNet concept URI"""
        if not isinstance(concept_str, str):
            return "unknown"
            
        parts = concept_str.split('/')
        if len(parts) >= 4:
            # Format is typically /c/LANG/CONCEPT
            return parts[2]
        return "unknown"
    
    def parse_weight(self, weight_str):
        """Parse weight JSON string to extract numeric weight"""
        if not isinstance(weight_str, str):
            return 1.0
            
        try:
            weight_data = json.loads(weight_str)
            # ConceptNet weights are typically in 'weight' field
            return float(weight_data.get('weight', 1.0))
        except:
            return 1.0
    
    def build_semantic_graph(self, max_concepts=200, min_weight=1.0, sample_size=0.25):
        """Build semantic graph from ConceptNet data"""
        print("Building semantic graph from ConceptNet data...")
        
        if self.english_data is None and self.german_data is None:
            print("No ConceptNet data provided.")
            return
        
        # Combine datasets
        all_data = []
        if self.english_data is not None:
            print(f"Processing {len(self.english_data)} English ConceptNet assertions...")
            sample_size_en = int(len(self.english_data) * sample_size)
            print(f"Will sample {sample_size_en} English assertions")
            all_data.append(('en', self.english_data))
        
        if self.german_data is not None:
            print(f"Processing {len(self.german_data)} German ConceptNet assertions...")
            sample_size_de = int(len(self.german_data) * sample_size)
            print(f"Will sample {sample_size_de} German assertions")
            all_data.append(('de', self.german_data))
        
        # Track concepts and their occurrence count
        concept_counts = {}
        
        # Process each language dataset
        for lang, data in all_data:
            curr_sample_size = int(len(data) * sample_size)
            data_sample = data.sample(n=curr_sample_size, random_state=42)
            print(f"Sampling {curr_sample_size} assertions from {len(data)} {lang} assertions")
            
            # Process assertions
            for _, row in tqdm(data_sample.iterrows(), desc=f"Processing {lang} assertions", total=len(data_sample)):
                try:
                    # Extract source and target concepts
                    source_concept = self.clean_concept_name(row['start'])
                    target_concept = self.clean_concept_name(row['end'])
                    
                    # Extract relation type
                    relation_type = self.extract_relation_type(row['rel'])
                    self.relation_types.add(relation_type)
                    
                    # Extract languages
                    source_lang = self.extract_language(row['start'])
                    target_lang = self.extract_language(row['end'])
                    
                    # Parse weight
                    weight = self.parse_weight(row['weight'])
                    
                    # Skip low-weight relationships
                    if weight < min_weight:
                        continue
                    
                    # Track concept occurrences
                    concept_counts[source_concept] = concept_counts.get(source_concept, 0) + 1
                    concept_counts[target_concept] = concept_counts.get(target_concept, 0) + 1
                    
                    # Add to graph
                    self.semantic_graph.add_node(
                        source_concept,
                        lang=source_lang,
                        count=concept_counts[source_concept]
                    )
                    
                    self.semantic_graph.add_node(
                        target_concept,
                        lang=target_lang,
                        count=concept_counts[target_concept]
                    )
                    
                    # Add edge with relation data
                    self.semantic_graph.add_edge(
                        source_concept,
                        target_concept,
                        relation=relation_type,
                        weight=weight
                    )
                    
                except Exception as e:
                    warnings.warn(f"Error processing assertion: {e}")
        
        # Limit to top concepts if needed
        if len(concept_counts) > max_concepts:
            print(f"Limiting graph to top {max_concepts} concepts...")
            top_concepts = sorted(concept_counts.items(), key=lambda x: x[1], reverse=True)[:max_concepts]
            top_concept_names = {c[0] for c in top_concepts}
            
            # Create subgraph with only top concepts
            subgraph = nx.DiGraph()
            
            for node in top_concept_names:
                if self.semantic_graph.has_node(node):
                    subgraph.add_node(
                        node,
                        **self.semantic_graph.nodes[node]
                    )
            
            for source, target, data in self.semantic_graph.edges(data=True):
                if source in top_concept_names and target in top_concept_names:
                    subgraph.add_edge(
                        source,
                        target,
                        **data
                    )
            
            self.semantic_graph = subgraph
        
        print(f"Semantic graph built with {self.semantic_graph.number_of_nodes()} nodes and {self.semantic_graph.number_of_edges()} edges")
        
        # Infer semantic categories
        self.infer_semantic_categories()
        
        return self.semantic_graph
    
    def infer_semantic_categories(self):
        """Infer semantic categories for concepts based on relationships"""
        print("Inferring semantic categories...")
        categories = {}
        
        # Count relationship types for each concept
        for node in self.semantic_graph.nodes():
            # Initialize as generic
            categories[node] = 'generic'
            
            # Get all relationships involving this concept
            in_edges = self.semantic_graph.in_edges(node, data=True)
            out_edges = self.semantic_graph.out_edges(node, data=True)
            
            # Count relationship types
            person_relations = 0
            place_relations = 0
            animal_relations = 0
            
            for _, _, data in in_edges:
                rel = data.get('relation', '')
                if rel in {'IsA/person', 'CapableOf', 'HasA'}:
                    person_relations += 1
                elif rel in {'AtLocation', 'LocatedNear', 'HasA'}:
                    place_relations += 1
                elif rel in {'IsA/animal', 'CapableOf'}:
                    animal_relations += 1
            
            for _, _, data in out_edges:
                rel = data.get('relation', '')
                if rel in {'IsA/person', 'CapableOf', 'HasA'}:
                    person_relations += 1
                elif rel in {'AtLocation', 'LocatedNear', 'HasA'}:
                    place_relations += 1
                elif rel in {'IsA/animal', 'CapableOf'}:
                    animal_relations += 1
            
            # Assign category based on dominant relationships
            max_relations = max(person_relations, place_relations, animal_relations)
            if max_relations > 0:
                if max_relations == person_relations:
                    categories[node] = 'person'
                elif max_relations == place_relations:
                    categories[node] = 'place'
                elif max_relations == animal_relations:
                    categories[node] = 'animal'
        
        # Update graph with categories
        nx.set_node_attributes(self.semantic_graph, categories, 'category')
        
        # Print category statistics
        category_counts = {}
        for cat in categories.values():
            category_counts[cat] = category_counts.get(cat, 0) + 1
        print(f"Inferred categories: {category_counts}")
    
    def compute_important_relationships(self, threshold=0.5, max_relationships=30):
        """Compute the most important relationships between concepts based on vector similarity"""
        important_relationships = []
        
        # Get all pairs of concepts
        concepts = list(self.concept_vectors.keys())
        for i, concept1 in enumerate(concepts):
            for concept2 in concepts[i+1:]:
                # Get vectors
                vec1 = self.concept_vectors[concept1]['vector']
                vec2 = self.concept_vectors[concept2]['vector']
                
                # Compute cosine similarity
                similarity = 1 - cosine(vec1, vec2)
                
                if similarity > threshold:
                    important_relationships.append({
                        'source': concept1,
                        'target': concept2,
                        'similarity': similarity
                    })
        
        # Sort by similarity and take top N
        important_relationships.sort(key=lambda x: x['similarity'], reverse=True)
        return important_relationships[:max_relationships]
    
    def generate_concept_vectors(self, dimensions=5):
        """Generate concept vectors based on graph structure"""
        print(f"Generating {dimensions}-dimensional concept vectors...")
        
        # Use node2vec or similar embedding
        nodes = list(self.semantic_graph.nodes())
        
        # Simple embedding based on connectivity patterns
        adjacency_matrix = nx.adjacency_matrix(self.semantic_graph).todense()
        
        # Use SVD to reduce dimensionality
        U, _, _ = np.linalg.svd(adjacency_matrix)
        embeddings = U[:, :dimensions]
        
        # Normalize embeddings
        embeddings = (embeddings - embeddings.mean(axis=0)) / embeddings.std(axis=0)
        
        # Convert to dictionary with structured data
        for i, node in enumerate(nodes):
            self.concept_vectors[node] = {
                'vector': embeddings[i],
                'category': self.semantic_graph.nodes[node].get('category', 'generic')
            }
        
        print(f"Generated vectors for {len(self.concept_vectors)} concepts")
        return self.concept_vectors

In [25]:

class SemanticVisualizer:
    def __init__(self, concept_processor=None):
        self.concept_processor = concept_processor
        self.fig = None
        self.ax = None
        self.category_colors = {
            'person': '#FF6B6B',  # Warm red
            'place': '#4ECDC4',   # Teal
            'animal': '#FFD93D',  # Bright yellow
            'generic': '#95A5A6'  # Neutral gray
        }
        self.category_descriptions = {
            'person': 'Human Entities & Roles',
            'place': 'Locations & Spaces',
            'animal': 'Living Creatures',
            'generic': 'Abstract Concepts'
        }

    def visualize_concepts_improved(self, output_dir, num_frames=30):
        """Create an enhanced visualization with labels, legends, and smooth transitions"""
        try:
            print("Creating enhanced semantic visualization...")
            
            if not self.concept_processor or not self.concept_processor.concept_vectors:
                raise ValueError("Concept processor not initialized or no vectors generated")
            
            # Get important relationships
            try:
                important_relationships = self.concept_processor.compute_important_relationships(
                    threshold=0.5, 
                    max_relationships=30
                )
            except Exception as e:
                print(f"Warning: Could not compute relationships: {str(e)}")
                important_relationships = []

            # Setup figure with high DPI for crisp text
            plt.close('all')
            fig = plt.figure(figsize=(20, 14), facecolor='black', dpi=150)
            ax = fig.add_subplot(111, projection='3d', facecolor='black')
            
            # Add a title with project info
            fig.suptitle('Semantic Concept Space Visualization\nRelational Semantic Convergence (RSC) Theory', 
                        color='white', y=0.95, fontsize=16, fontweight='bold')

            def update(frame):
                try:
                    ax.clear()
                    ax.set_facecolor('black')
                    
                    # Configure axis appearance
                    ax.grid(True, alpha=0.1, color='white')
                    ax.xaxis.pane.fill = False
                    ax.yaxis.pane.fill = False
                    ax.zaxis.pane.fill = False
                    
                    # Remove axis labels but keep tick marks
                    ax.set_xticklabels([])
                    ax.set_yticklabels([])
                    ax.set_zticklabels([])
                    
                    # Calculate smooth rotation angle
                    theta = (frame / num_frames) * 2 * np.pi
                    
                    # Convert concept vectors to 3D points with rotation
                    points = {}
                    for concept, data in self.concept_processor.concept_vectors.items():
                        vector = data['vector'][:3]
                        
                        # Apply smooth rotation matrix
                        x, y, z = vector
                        x_rot = x * np.cos(theta) - y * np.sin(theta)
                        y_rot = x * np.sin(theta) + y * np.cos(theta)
                        
                        points[concept] = (x_rot, y_rot, z)
                    
                    # Plot points by category with enhanced visual elements
                    legend_elements = []
                    for category, color in self.category_colors.items():
                        cat_points = [
                            (concept, (x, y, z)) for concept, (x, y, z) in points.items()
                            if self.concept_processor.concept_vectors[concept]['category'] == category
                        ]
                        
                        if cat_points:
                            concepts, coords = zip(*cat_points)
                            xs, ys, zs = zip(*coords)
                            
                            # Create scatter plot with glowing effect
                            scatter = ax.scatter(xs, ys, zs, 
                                              c=color, 
                                              alpha=0.8, 
                                              s=100,  # Larger points
                                              edgecolors='white',
                                              linewidth=0.5)
                            
                            # Add category to legend
                            legend_elements.append(plt.Line2D([0], [0], 
                                                            marker='o', 
                                                            color='none',
                                                            markerfacecolor=color,
                                                            markeredgecolor='white',
                                                            markersize=10,
                                                            label=self.category_descriptions[category]))
                            
                            # Add labels for important concepts
                            for concept, (x, y, z) in zip(concepts, coords):
                                if len(concept) > 2:  # Only label non-trivial concepts
                                    ax.text(x, y, z, 
                                          concept,
                                          color='white',
                                          fontsize=8,
                                          alpha=0.7,
                                          backgroundcolor=(0, 0, 0, 0.3))
                    
                    # Add connections between related concepts
                    if important_relationships and frame == 0:  # Only on first frame for performance
                        for rel in important_relationships[:10]:  # Limit to top 10 relationships
                            if rel['source'] in points and rel['target'] in points:
                                x1, y1, z1 = points[rel['source']]
                                x2, y2, z2 = points[rel['target']]
                                ax.plot([x1, x2], [y1, y2], [z1, z2], 
                                      color='white',
                                      alpha=0.2,
                                      linestyle='--')
                    
                    # Add legend with enhanced styling
                    legend = ax.legend(handles=legend_elements,
                                     loc='center left',
                                     bbox_to_anchor=(1.15, 0.5),
                                     title='Semantic Categories',
                                     facecolor='black',
                                     edgecolor='white',
                                     framealpha=0.8)
                    legend.get_title().set_color('white')
                    for text in legend.get_texts():
                        text.set_color('white')
                    
                    # Add RSC theory info
                    ax.text2D(0.02, 0.02, 
                             'RSC Theory Visualization\nShowing semantic relationships and concept clustering',
                             transform=ax.transAxes,
                             color='white',
                             alpha=0.7,
                             fontsize=8)
                    
                    return ax
                
                except Exception as e:
                    print(f"Error in update frame {frame}: {str(e)}")
                    return ax

            # Create animation with enhanced parameters
            anim = animation.FuncAnimation(
                fig, 
                update,
                frames=num_frames,
                interval=50,  # Faster frame rate for smoother animation
                blit=False
            )
            
            # Create a static preview image
            print("Creating enhanced static preview...")
            update(0)
            
            # Save with high quality settings
            static_path = os.path.join(output_dir, f'semantica_readable_preview_{uuid.uuid4()}.png')
            plt.savefig(static_path, dpi=150, bbox_inches='tight')
            
            # Save animation with enhanced quality
            output_path = os.path.join(output_dir, f'semantica_readable_{uuid.uuid4()}.gif')
            # Use a more robust method to save the animation
            frames = []
            frame_files = []  # Keep track of temporary files
            tmp_dir = os.path.join(output_dir, 'tmp_frames')
            
            # Create temporary directory for frames
            if not os.path.exists(tmp_dir):
                os.makedirs(tmp_dir)
                
            try:
                print("Generating frames...")
                for i in range(num_frames):
                    # Update the figure for this frame
                    update(i)
                    
                    # Save frame to a temporary file instead of keeping in memory
                    tmp_file = os.path.join(tmp_dir, f'frame_{i:04d}.png')
                    plt.savefig(tmp_file, dpi=150, bbox_inches='tight',
                               facecolor=self.bg_gradient['bottom'])
                    frame_files.append(tmp_file)
                    
                    # Print progress
                    if i % 10 == 0 or i == num_frames - 1:
                        print(f"Generated frame {i+1}/{num_frames}")
                
                # Create GIF using external library
                print("Saving animation...")
                from PIL import Image
                
                # Load all frames from files
                frames = [Image.open(f) for f in frame_files]
                
                # Ensure all frames are the same size
                if frames:
                    size = frames[0].size
                    frames = [f.resize(size) for f in frames]
                    
                    # Save as GIF
                    frames[0].save(
                        output_path,
                        save_all=True,
                        append_images=frames[1:],
                        optimize=False,
                        duration=1000/30,  # 30 FPS
                        loop=0
                    )
                    
                    # Close all frames
                    for f in frames:
                        f.close()
                    
                    print(f"Enhanced visualization saved successfully to {output_path}")
            finally:
                # Clean up temporary files
                for file in frame_files:
                    try:
                        if os.path.exists(file):
                            os.remove(file)
                    except Exception as e:
                        print(f"Warning: Could not remove temporary file {file}: {e}")
                        
                # Try to remove temp directory
                try:
                    if os.path.exists(tmp_dir):
                        os.rmdir(tmp_dir)
                except Exception as e:
                    print(f"Warning: Could not remove temporary directory: {e}")
            
            plt.close()
            return output_path
            
        except Exception as e:
            print(f"Error creating visualization: {str(e)}")
            traceback.print_exc()
            return None

In [26]:
# Initialize processor with the datasets we loaded at the beginning
print("Initializing ConceptNetProcessor with loaded datasets...")
processor = ConceptNetProcessor(english_conceptnet, german_conceptnet)

# Build semantic graph with 25% sampling
processor.build_semantic_graph(
    max_concepts=100,  # Keep reasonable number of concepts for visualization
    min_weight=1.0,
    sample_size=0.05  # Use 25% of the data
)

# Generate concept vectors
processor.generate_concept_vectors(dimensions=5)

# Initialize visualizer
visualizer = SemanticVisualizer(processor)

# Create improved visualization with more frames for smoother animation
output_path = visualizer.visualize_concepts_improved(
    output_dir,
    num_frames=300  # Increased number of frames for smoother animation
)

print(f"Visualization created at: {output_path}")

Initializing ConceptNetProcessor with loaded datasets...
ConceptNetProcessor initialized
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 171150 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 53947 German assertions
Sampling 171150 assertions from 3423004 en assertions


Processing en assertions: 100%|██████████| 171150/171150 [00:04<00:00, 39983.64it/s]



Sampling 53947 assertions from 1078946 de assertions


Processing de assertions: 100%|██████████| 53947/53947 [00:01<00:00, 36625.40it/s]



Limiting graph to top 100 concepts...
Semantic graph built with 100 nodes and 697 edges
Inferring semantic categories...
Inferred categories: {'generic': 100}
Generating 5-dimensional concept vectors...
Generated vectors for 100 concepts
Creating enhanced semantic visualization...
Creating enhanced static preview...
Generating frames...
Error creating visualization: 'SemanticVisualizer' object has no attribute 'bg_gradient'
Visualization created at: None
Generating frames...
Error creating visualization: 'SemanticVisualizer' object has no attribute 'bg_gradient'
Visualization created at: None


Traceback (most recent call last):
  File "C:\Users\Erich Curtis\AppData\Local\Temp\ipykernel_61348\574494523.py", line 190, in visualize_concepts_improved
    facecolor=self.bg_gradient['bottom'])
              ^^^^^^^^^^^^^^^^
AttributeError: 'SemanticVisualizer' object has no attribute 'bg_gradient'


# Phase 2: Enhanced Visualization

Based on feedback, we're improving the visualization to make it more intuitive and appealing for both technical and general audiences. Key enhancements include:

1. **Progress indicator** - Shows current frame and total frames so viewers know where they are in the animation
2. **Improved visual relationships** - Clear connection lines between related concepts with directional arrows
3. **Interactive elements** - Hover tooltips and concept highlighting (for interactive environments)
4. **Better storytelling** - Explanatory text and conceptual grouping visualizations
5. **Visual appeal** - Enhanced color scheme, lighting effects, and background gradients
6. **Clearer concept representation** - Size nodes based on importance, add labels for key concepts
7. **Animation transitions** - Smoother transitions and camera movements between frames

In [32]:
class EnhancedSemanticVisualizer:
    def __init__(self, concept_processor=None):
        self.concept_processor = concept_processor
        self.fig = None
        self.ax = None
        # Enhanced color palette with deeper, more vibrant colors
        self.category_colors = {
            'person': '#FF5E7E',     # Vibrant pink/red
            'place': '#36EAFF',      # Bright cyan
            'animal': '#FFDD3C',     # Golden yellow
            'generic': '#B8C6DB'     # Soft lavender
        }
        # More descriptive category labels
        self.category_descriptions = {
            'person': 'Human Concepts & Relationships',
            'place': 'Spatial & Location Concepts',
            'animal': 'Living Entities & Nature',
            'generic': 'Abstract & General Concepts'
        }
        # Background gradient colors
        self.bg_gradient = {
            'top': '#0F2027',       # Dark blue-black
            'middle': '#203A43',    # Deep teal
            'bottom': '#2C5364'     # Navy blue
        }
        # Store background rectangles
        self.bg_rects = []
        # Store progress indicator elements
        self.progress_elements = []
        
    def _create_background_gradient(self, fig, ax):
        """Create a beautiful gradient background"""
        # Clear any existing background rectangles
        for rect in self.bg_rects:
            if rect in fig.patches:
                fig.patches.remove(rect)
        self.bg_rects = []
        
        # Create new background rectangles
        bottom_rect = plt.Rectangle(
            (0, 0), 1, 1,
            transform=fig.transFigure,
            color=self.bg_gradient['bottom'],
            alpha=0.9, zorder=-1
        )
        middle_rect = plt.Rectangle(
            (0, 0.3), 1, 0.4,
            transform=fig.transFigure,
            color=self.bg_gradient['middle'],
            alpha=0.7, zorder=-1
        )
        top_rect = plt.Rectangle(
            (0, 0.7), 1, 0.3,
            transform=fig.transFigure,
            color=self.bg_gradient['top'],
            alpha=0.8, zorder=-1
        )
        
        # Add rectangles to figure
        fig.patches.extend([bottom_rect, middle_rect, top_rect])
        self.bg_rects = [bottom_rect, middle_rect, top_rect]
        
        return fig
    
    def _add_progress_indicator(self, fig, frame, total_frames):
        """Add a visual progress indicator showing current frame and total"""
        # Clear previous progress indicator elements
        for element in self.progress_elements:
            if isinstance(element, matplotlib.text.Text) and element in fig.texts:
                fig.texts.remove(element)
            elif isinstance(element, matplotlib.patches.Rectangle) and element in fig.patches:
                fig.patches.remove(element)
        self.progress_elements = []
                
        # Calculate percentage complete
        percentage = (frame / total_frames) * 100
        
        # Create progress text
        progress_text = fig.text(
            0.01, 0.01,
            f"Frame: {frame+1}/{total_frames} ({percentage:.1f}%)",
            color='white',
            fontsize=10,
            alpha=0.8,
            transform=fig.transFigure
        )
        self.progress_elements.append(progress_text)
        
        # Add progress bar background
        progress_bar_bg = plt.Rectangle(
            (0.01, 0.03), 0.2, 0.01,
            transform=fig.transFigure,
            color='white', alpha=0.3
        )
        fig.patches.append(progress_bar_bg)
        self.progress_elements.append(progress_bar_bg)
        
        # Add progress bar fill
        progress_bar_fill = plt.Rectangle(
            (0.01, 0.03), 0.2 * (frame / total_frames), 0.01,
            transform=fig.transFigure,
            color='#36EAFF', alpha=0.8
        )
        fig.patches.append(progress_bar_fill)
        self.progress_elements.append(progress_bar_fill)
        
        return fig
    
    def _add_relationship_lines(self, ax, points, important_relationships, frame_pct):
        """Add visually appealing relationship lines between concepts"""
        # Only show the top relationships for clarity
        if not important_relationships:
            return ax
            
        # Create a pulsing effect based on frame percentage
        pulse = 0.5 + 0.5 * np.sin(frame_pct * 2 * np.pi * 2)  # 2 pulses per cycle
        
        # Draw connections with gradient colors and pulse effect
        for i, rel in enumerate(important_relationships[:15]):  # Limit to top relationships
            if rel['source'] in points and rel['target'] in points:
                src = points[rel['source']]
                tgt = points[rel['target']]
                
                # Gradually reveal relationships throughout the animation
                reveal_threshold = i / 15  # Stagger appearance
                if frame_pct < reveal_threshold:
                    continue
                    
                # Calculate line alpha based on similarity strength and pulse
                alpha = min(0.7, rel['similarity'] * pulse)
                
                # Determine relationship color based on similarity
                if rel['similarity'] > 0.8:
                    color = '#FF5E7E'  # Strong relationship - pink
                elif rel['similarity'] > 0.65:
                    color = '#36EAFF'  # Medium relationship - cyan
                else:
                    color = '#FFFFFF'  # Weak relationship - white
                    
                # Draw the relationship line with gradient effect
                line = ax.plot([src[0], tgt[0]], [src[1], tgt[1]], [src[2], tgt[2]],
                      color=color,
                      alpha=alpha,
                      linewidth=1.5,
                      zorder=1)
                
                # Add small arrow to indicate relationship direction
                if rel['similarity'] > 0.7:  # Only for stronger relationships
                    # Calculate midpoint with slight offset toward target
                    midpoint = tuple(0.6*src[i] + 0.4*tgt[i] for i in range(3))
                    
                    # Draw small sphere at midpoint to represent connection type
                    ax.scatter([midpoint[0]], [midpoint[1]], [midpoint[2]],
                             color=color,
                             s=30,
                             alpha=alpha*1.2,
                             edgecolors='white',
                             linewidth=0.5)
        
        return ax
        
    def _add_glow_effect(self, ax, xs, ys, zs, color, size):
        """Add a subtle glow effect to make points more visually appealing"""
        # Create glow by adding multiple layers of decreasing opacity
        for glow_size, alpha in [(size*3, 0.03), (size*2, 0.05), (size*1.5, 0.1)]:
            ax.scatter(xs, ys, zs,
                     color=color,
                     s=glow_size,
                     alpha=alpha,
                     edgecolors='none')
        
        return ax
    
    def _add_explanatory_elements(self, fig, ax, frame_pct):
        """Add explanatory text and visual elements to guide understanding"""
        # Title with project info and frame context
        title_txt = fig.text(
            0.5, 0.95,
            'Semantic Concept Space Visualization',
            ha='center',
            color='white',
            fontsize=18,
            fontweight='bold',
            alpha=0.9,
            transform=fig.transFigure
        )
        
        # Subtitle with RSC theory explanation
        subtitle_txt = fig.text(
            0.5, 0.92,
            'Relational Semantic Convergence (RSC) Theory',
            ha='center',
            color='white',
            fontsize=14,
            fontweight='normal',
            alpha=0.8,
            transform=fig.transFigure
        )
        
        # Rotating explanation text that changes throughout the animation
        explanations = [
            "Visualizing how concepts form relationships in semantic space",
            "Connected concepts share semantic properties and relationships",
            "Colored clusters represent different semantic categories",
            "Discover how meaning emerges from conceptual connections"
        ]
        
        # Select explanation based on frame position
        explanation_idx = int(frame_pct * len(explanations)) % len(explanations)
        explanation_txt = fig.text(
            0.5, 0.89,
            explanations[explanation_idx],
            ha='center',
            color='white',
            fontsize=11,
            fontstyle='italic',
            alpha=0.7,
            transform=fig.transFigure
        )
        
        # Add a visual indicator for rotation direction
        arrow_angle = frame_pct * 2 * np.pi
        arrow_x = 0.97 + 0.02 * np.cos(arrow_angle)
        arrow_y = 0.5 + 0.02 * np.sin(arrow_angle)
        rotation_indicator = fig.text(
            0.97, 0.5,
            '↻',  # Rotation symbol
            ha='center',
            va='center',
            color='white',
            fontsize=14,
            alpha=0.6,
            transform=fig.transFigure
        )
        
        # Add copyright and attribution
        fig.text(
            0.99, 0.01,
            '© Semantica RSC Project ' + time.strftime('%Y'),
            ha='right',
            color='white',
            fontsize=8,
            alpha=0.5,
            transform=fig.transFigure
        )
        
        return fig
    
    def _add_visual_cues(self, ax, points, frame_pct):
        """Add visual cues to highlight important features"""
        # Find the most central points to highlight
        center = np.array([0, 0, 0])
        distances = {concept: np.linalg.norm(np.array(pos) - center) 
                    for concept, pos in points.items()}
        
        # Get 3 closest points to center
        central_concepts = sorted(distances.items(), key=lambda x: x[1])[:3]
        
        # Highlight these central concepts with pulsing focus rings
        pulse = 0.5 + 0.5 * np.sin(frame_pct * 2 * np.pi * 3)  # 3 pulses per cycle
        
        for concept, distance in central_concepts:
            if concept in points:
                x, y, z = points[concept]
                
                # Draw focus ring
                theta = np.linspace(0, 2*np.pi, 20)
                radius = 0.2 + 0.05 * pulse
                
                # Create a small circle around the point
                circle_x = x + radius * np.cos(theta)
                circle_y = y + radius * np.sin(theta)
                circle_z = np.full_like(theta, z)
                
                ax.plot(circle_x, circle_y, circle_z, 
                      color='white', 
                      alpha=0.3*pulse,
                      linewidth=1)
        
        return ax
    
    def visualize_concepts_enhanced(self, output_dir, num_frames=60):
        """Create a highly enhanced visualization with improved visual appeal and clarity"""
        try:
            print("Creating enhanced semantic visualization...")
            
            if not self.concept_processor or not self.concept_processor.concept_vectors:
                raise ValueError("Concept processor not initialized or no vectors generated")
            
            # Get important relationships
            try:
                important_relationships = self.concept_processor.compute_important_relationships(
                    threshold=0.5, 
                    max_relationships=30
                )
            except Exception as e:
                print(f"Warning: Could not compute relationships: {str(e)}")
                important_relationships = []

            # Setup figure with high DPI for crisp text
            plt.close('all')
            fig = plt.figure(figsize=(16, 12), dpi=150)
            ax = fig.add_subplot(111, projection='3d')
            
            # Apply the background gradient
            self._create_background_gradient(fig, ax)
            
            # Calculate node importance based on connectivity
            node_importance = {}
            for concept in self.concept_processor.concept_vectors.keys():
                # Count occurrences in important relationships
                count = sum(1 for rel in important_relationships 
                           if rel['source'] == concept or rel['target'] == concept)
                node_importance[concept] = 30 + (count * 20)  # Base size + importance factor
            
            # Lists to keep track of text objects
            text_objects = []
            
            def update(frame):
                try:
                    # Clear previous frame
                    ax.clear()
                    
                    # Handle text objects removal more safely
                    for txt in fig.texts[:]:
                        fig.texts.remove(txt)
                    
                    # Calculate frame percentage for animations
                    frame_pct = frame / num_frames
                    
                    # Configure axis appearance
                    ax.grid(False)  # Remove grid for cleaner look
                    ax.xaxis.pane.fill = False
                    ax.yaxis.pane.fill = False
                    ax.zaxis.pane.fill = False
                    
                    # Make panes completely transparent
                    ax.xaxis.pane.set_edgecolor('none')
                    ax.yaxis.pane.set_edgecolor('none')
                    ax.zaxis.pane.set_edgecolor('none')
                    
                    # Remove axis labels and ticks for cleaner look
                    ax.set_xticklabels([])
                    ax.set_yticklabels([])
                    ax.set_zticklabels([])
                    ax.set_xticks([])
                    ax.set_yticks([])
                    ax.set_zticks([])
                    
                    # Calculate dynamic camera angle for smoother rotation
                    theta = frame_pct * 2 * np.pi
                    phi = 0.2 * np.sin(frame_pct * 2 * np.pi * 0.5) + 0.3  # Gentle up/down motion
                    
                    # Set viewing angle with smooth transitions
                    ax.view_init(30 + 5 * np.sin(phi), 45 + 180 * frame_pct)
                    
                    # Convert concept vectors to 3D points with dynamic positioning
                    points = {}
                    for concept, data in self.concept_processor.concept_vectors.items():
                        vector = data['vector'][:3]
                        
                        # Apply dynamic rotation matrix
                        x, y, z = vector
                        x_rot = x * np.cos(theta) - y * np.sin(theta)
                        y_rot = x * np.sin(theta) + y * np.cos(theta)
                        z_rot = z + 0.05 * np.sin(theta * 2 + x)  # Add gentle wave motion
                        
                        points[concept] = (x_rot, y_rot, z_rot)
                    
                    # Add relationship lines first (so they appear behind points)
                    self._add_relationship_lines(ax, points, important_relationships, frame_pct)
                    
                    # Plot points by category with enhanced visual elements
                    legend_elements = []
                    for category, color in self.category_colors.items():
                        cat_points = [
                            (concept, points[concept], node_importance[concept]) 
                            for concept in self.concept_processor.concept_vectors
                            if concept in points and 
                            self.concept_processor.concept_vectors[concept]['category'] == category
                        ]
                        
                        if cat_points:
                            # Split the data for plotting
                            concepts, coords, sizes = zip(*cat_points)
                            xs, ys, zs = zip(*coords)
                            
                            # Add glow effect for nicer visuals
                            self._add_glow_effect(ax, xs, ys, zs, color, 30)
                            
                            # Create scatter plot with improved styling
                            scatter = ax.scatter(xs, ys, zs, 
                                              c=color, 
                                              alpha=0.9, 
                                              s=sizes,  # Size based on importance
                                              edgecolors='white',
                                              linewidth=0.5,
                                              zorder=10)
                            
                            # Add category to legend with improved styling
                            legend_elements.append(plt.Line2D(
                                [0], [0], 
                                marker='o', 
                                color='none',
                                markerfacecolor=color,
                                markeredgecolor='white',
                                markersize=10,
                                label=self.category_descriptions[category]))
                            
                            # Add labels for important concepts with improved styling
                            for concept, (x, y, z), size in zip(concepts, coords, sizes):
                                # Only label significant concepts for clarity
                                if len(concept) > 2 and size > 50:  # Important nodes with meaningful names
                                    # Calculate label transparency based on size
                                    label_alpha = min(0.9, size / 100)
                                    
                                    # Create more visible text with better contrast
                                    ax.text(x, y, z + 0.05,  # Slight offset
                                          concept,
                                          color='white',
                                          fontsize=8,
                                          fontweight='bold',
                                          alpha=label_alpha,
                                          backgroundcolor=(0, 0, 0, 0.4),
                                          ha='center',
                                          zorder=20)
                    
                    # Add visual cues to highlight key areas
                    self._add_visual_cues(ax, points, frame_pct)
                    
                    # Add legend with enhanced styling
                    legend = ax.legend(handles=legend_elements,
                                     loc='upper right',
                                     bbox_to_anchor=(0.99, 0.99),
                                     title='Semantic Categories',
                                     facecolor=(0.1, 0.1, 0.1, 0.7),
                                     edgecolor='white',
                                     fontsize=10)
                    
                    if legend:
                        legend.get_title().set_color('white')
                        for text in legend.get_texts():
                            text.set_color('white')
                    
                    # Add progress indicator
                    self._add_progress_indicator(fig, frame, num_frames)
                    
                    # Add explanatory text and elements
                    self._add_explanatory_elements(fig, ax, frame_pct)
                    
                    return fig
                
                except Exception as e:
                    print(f"Error in update frame {frame}: {str(e)}")
                    traceback.print_exc()
                    return fig

            # Create animation with enhanced parameters
            anim = animation.FuncAnimation(
                fig, 
                update,
                frames=num_frames,
                interval=40,  # Faster frame rate for smoother animation
                blit=False
            )
            
            # Create a static preview image
            print("Creating enhanced static preview...")
            update(0)
            
            # Save with high quality settings
            static_path = os.path.join(output_dir, f'semantica_enhanced_preview_{uuid.uuid4()}.png')
            plt.savefig(static_path, dpi=150, bbox_inches='tight')
            print(f"Preview image saved to: {static_path}")
            
            # Save animation with enhanced quality
            output_path = os.path.join(output_dir, f'semantica_enhanced_{uuid.uuid4()}.gif')
            print(f"Generating animation (this may take some time)...")
            
            # Add a timestamp to the animation
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            fig.text(
                0.5, 0.01,
                f"Generated: {timestamp}",
                ha='center',
                color='white',
                fontsize=8,
                alpha=0.6,
                transform=fig.transFigure
            )
            
            # Use a more robust method to save the animation
            frames = []
            frame_files = []  # Keep track of temporary files
            tmp_dir = os.path.join(output_dir, 'tmp_frames')
            
            # Create temporary directory for frames
            if not os.path.exists(tmp_dir):
                os.makedirs(tmp_dir)
                
            try:
                print("Generating frames...")
                for i in range(num_frames):
                    # Update the figure for this frame
                    update(i)
                    
                    # Save frame to a temporary file instead of keeping in memory
                    tmp_file = os.path.join(tmp_dir, f'frame_{i:04d}.png')
                    plt.savefig(tmp_file, dpi=150, bbox_inches='tight',
                               facecolor=self.bg_gradient['bottom'])
                    frame_files.append(tmp_file)
                    
                    # Print progress
                    if i % 10 == 0 or i == num_frames - 1:
                        print(f"Generated frame {i+1}/{num_frames}")
                
                # Create GIF using external library
                print("Saving animation...")
                from PIL import Image
                
                # Load all frames from files
                frames = [Image.open(f) for f in frame_files]
                
                # Ensure all frames are the same size
                if frames:
                    size = frames[0].size
                    frames = [f.resize(size) for f in frames]
                    
                    # Save as GIF
                    frames[0].save(
                        output_path,
                        save_all=True,
                        append_images=frames[1:],
                        optimize=False,
                        duration=1000/30,  # 30 FPS
                        loop=0
                    )
                    
                    # Close all frames
                    for f in frames:
                        f.close()
                    
                    print(f"Enhanced visualization saved successfully to {output_path}")
            finally:
                # Clean up temporary files
                for file in frame_files:
                    try:
                        if os.path.exists(file):
                            os.remove(file)
                    except Exception as e:
                        print(f"Warning: Could not remove temporary file {file}: {e}")
                        
                # Try to remove temp directory
                try:
                    if os.path.exists(tmp_dir):
                        os.rmdir(tmp_dir)
                except Exception as e:
                    print(f"Warning: Could not remove temporary directory: {e}")
            
            plt.close()
            return output_path
            
        except Exception as e:
            print(f"Error creating visualization: {str(e)}")
            traceback.print_exc()
            return None

In [33]:
# Phase 2: Generate the enhanced visualization
print("Starting Phase 2: Enhanced visualization generation...")

# Create enhanced visualizer with the same processor
enhanced_visualizer = EnhancedSemanticVisualizer(processor)




# Initialize processor with the datasets we loaded at the beginning
print("Initializing ConceptNetProcessor with loaded datasets...")
processor = ConceptNetProcessor(english_conceptnet, german_conceptnet)

# Build semantic graph with 25% sampling
processor.build_semantic_graph(
    max_concepts=10_000,  # Keep reasonable number of concepts for visualization
    min_weight=1.0,
    sample_size=0.5  # Use 25% of the data
)

# Generate concept vectors
processor.generate_concept_vectors(dimensions=5)

# Create the enhanced visualization with more frames for smoother animation
# and additional visual elements for clarity and appeal
output_path = enhanced_visualizer.visualize_concepts_enhanced(
    output_dir,
    num_frames=500  # Reasonable number of frames for smooth animation but faster rendering
)

print(f"Enhanced visualization created at: {output_path}")
print("\nKey improvements in Phase 2:")
print("1. Progress indicator showing current frame/total frames")
print("2. Beautiful gradient background instead of plain gray")
print("3. Clear relationship lines between related concepts")
print("4. Explanatory text elements that change throughout the animation")
print("5. Node sizing based on concept importance")
print("6. Improved color scheme and glow effects")
print("7. Smooth camera angles and transitions")
print("8. Visual cues highlighting important concepts")
print("9. Timestamp and attribution information")

Starting Phase 2: Enhanced visualization generation...
Initializing ConceptNetProcessor with loaded datasets...
ConceptNetProcessor initialized
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 1711502 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 539473 German assertions
Sampling 1711502 assertions from 3423004 en assertions
Sampling 1711502 assertions from 3423004 en assertions


Processing en assertions:  56%|█████▌    | 958006/1711502 [00:24<00:18, 41419.05it/s]

# Phase 3: Advanced Visualization Refinements

After reviewing Phase 2's visualization, we've identified several areas for further enhancement:

1. **Connection clarity** - Improve relationship line visibility and make the semantic connections clearer
2. **Text legibility** - Enhance contrast and positioning of labels for better readability
3. **Depth perception** - Add subtle depth cues to improve 3D space comprehension
4. **Camera dynamics** - Create more meaningful camera movements to highlight concept clusters
5. **Visual storytelling** - Add contextual information to explain the relationships between concepts
6. **Performance optimization** - Improve rendering efficiency for smoother animations
7. **Interactive elements** - Add visual responses to highlight related concepts when specific nodes are in focus

In [34]:
class AdvancedSemanticVisualizer(EnhancedSemanticVisualizer):
    def __init__(self, concept_processor=None):
        # Initialize parent class
        super().__init__(concept_processor)
        
        # Enhanced color palette with even better perceptual separation
        self.category_colors = {
            'person': '#FF3D7F',     # Brighter pink/red for better visibility
            'place': '#32CCFE',      # Clearer cyan
            'animal': '#FFCC00',     # More saturated gold
            'generic': '#C8D6E5'     # Lighter lavender for better contrast
        }
        
        # More descriptive category labels
        self.category_descriptions = {
            'person': 'Human Concepts & Relationships',
            'place': 'Spatial & Location Concepts',
            'animal': 'Living Entities & Natural World',
            'generic': 'Abstract & General Concepts'
        }
        
        # Improved background gradient colors for better depth perception
        self.bg_gradient = {
            'top': '#0A1921',       # Deeper dark blue for better contrast
            'middle': '#1C3342',    # Richer teal
            'bottom': '#2A4E60'     # Slightly brighter navy blue
        }
        
        # For tracking related concepts for interactive highlighting
        self.highlighted_concepts = set()
        
    def _add_depth_cues(self, ax, points):
        """Add subtle depth cues to enhance 3D perception"""
        # Create a subtle ground plane grid for better spatial orientation
        x_min, x_max = -1.5, 1.5
        y_min, y_max = -1.5, 1.5
        z_min = min(z for _, _, z in points.values()) - 0.1
        
        # Draw a subtle ground grid
        grid_alpha = 0.05
        grid_spacing = 0.2
        
        for x in np.arange(x_min, x_max + grid_spacing, grid_spacing):
            ax.plot([x, x], [y_min, y_max], [z_min, z_min], 
                   color='white', alpha=grid_alpha, linewidth=0.5)
                   
        for y in np.arange(y_min, y_max + grid_spacing, grid_spacing):
            ax.plot([x_min, x_max], [y, y], [z_min, z_min], 
                   color='white', alpha=grid_alpha, linewidth=0.5)
        
        # Add faint coordinate axes for reference
        axis_alpha = 0.15
        ax.plot([0, 1], [0, 0], [z_min, z_min], color='#FF5E7E', alpha=axis_alpha, linewidth=1)  # X-axis
        ax.plot([0, 0], [0, 1], [z_min, z_min], color='#36EAFF', alpha=axis_alpha, linewidth=1)  # Y-axis
        
        return ax
    
    def _enhance_relationship_lines(self, ax, points, important_relationships, frame_pct):
        """Enhanced version of relationship lines with better visual cues"""
        # Build on the parent class method
        ax = self._add_relationship_lines(ax, points, important_relationships, frame_pct)
        
        # Calculate dynamic opacity based on camera angle to improve visibility
        # as the visualization rotates
        theta = frame_pct * 2 * np.pi
        view_factor = abs(np.sin(theta))  # Increases visibility when viewing from certain angles
        
        # Add additional visual elements for the most important relationships
        top_relationships = sorted(
            [rel for rel in important_relationships if rel['source'] in points and rel['target'] in points],
            key=lambda x: x['similarity'], 
            reverse=True
        )[:5]  # Top 5 strongest relationships
        
        for rel in top_relationships:
            src = points[rel['source']]
            tgt = points[rel['target']]
            
            # Calculate midpoint for label placement
            midpoint = [(src[i] + tgt[i])/2 for i in range(3)]
            
            # Only show labels when the opacity is higher for readability
            if view_factor > 0.6 and rel['similarity'] > 0.75:
                # Add small connection label at midpoint
                ax.text(midpoint[0], midpoint[1], midpoint[2],
                      f"{rel['similarity']:.2f}",
                      color='white',
                      alpha=0.7 * view_factor,
                      fontsize=7,
                      ha='center',
                      bbox=dict(facecolor='black', alpha=0.4, pad=1),
                      zorder=15)
        
        return ax
    
    def _add_improved_glow_effect(self, ax, xs, ys, zs, color, sizes):
        """Add enhanced glow effect with size variation based on importance"""
        # Base glow effect from parent class
        ax = self._add_glow_effect(ax, xs, ys, zs, color, 30)
        
        # Add pulsing halos to the most important nodes
        # Find top 3 largest points (most important)
        if len(xs) > 3:
            top_indices = np.argsort(sizes)[-3:]
            
            for idx in top_indices:
                # Additional outer glow for important nodes
                ax.scatter([xs[idx]], [ys[idx]], [zs[idx]],
                         color=color,
                         s=sizes[idx] * 2.5,  # Much larger glow
                         alpha=0.05,         # Very subtle
                         edgecolors='none')
        
        return ax
        
    def _add_interactive_highlighting(self, ax, points, frame_pct, node_importance):
        """Add visual responses highlighting related concept groups"""
        # Simulate interactive highlighting by changing the focus over time
        highlight_rotation = int((frame_pct * 4) % 4)  # Rotate through 4 concept groups
        
        if highlight_rotation == 0:
            focus_category = 'person'
        elif highlight_rotation == 1:
            focus_category = 'place'
        elif highlight_rotation == 2:
            focus_category = 'animal'
        else:
            focus_category = 'generic'
            
        # Find concepts in the focus category
        focus_concepts = [
            concept for concept in self.concept_processor.concept_vectors
            if concept in points and 
            self.concept_processor.concept_vectors[concept]['category'] == focus_category
        ]
        
        # Only highlight if we have at least one concept in this category
        if not focus_concepts:
            return ax
            
        # Choose a single focus concept that changes over time
        idx = int((frame_pct * len(focus_concepts) * 10) % len(focus_concepts))
        focus_concept = focus_concepts[idx]
        
        # Draw a subtle highlight indicator
        if focus_concept in points:
            x, y, z = points[focus_concept]
            size = node_importance.get(focus_concept, 40)
            
            # Draw pulsing highlight
            pulse = 0.5 + 0.5 * np.sin(frame_pct * 2 * np.pi * 5)  # Fast pulse
            
            # Draw focus indicator
            ax.scatter([x], [y], [z],
                     color='white',
                     s=size * (1.5 + pulse * 0.5),
                     alpha=0.2 * pulse,
                     edgecolors='white',
                     linewidth=1)
            
            # Add subtle focus label
            category = self.concept_processor.concept_vectors[focus_concept]['category']
            color = self.category_colors[category]
            
            ax.text(x, y, z + 0.1,
                  f"↓ {focus_concept} ↓",
                  color=color,
                  fontsize=9,
                  fontweight='bold',
                  ha='center',
                  va='bottom',
                  alpha=0.8 * pulse,
                  bbox=dict(facecolor='black', alpha=0.4, pad=2),
                  zorder=100)
        
        return ax
    
    def _add_advanced_explanatory_elements(self, fig, ax, frame_pct):
        """Add more informative explanatory elements"""
        # Base explanatory elements from parent class
        fig = self._add_explanatory_elements(fig, ax, frame_pct)
        
        # Add concept count information
        concept_count = len(self.concept_processor.concept_vectors)
        
        # Add visualization statistics
        stats_text = fig.text(
            0.01, 0.07,
            f"Visualizing {concept_count} concepts\n"
            f"Dimensions: 3D projection of semantic space",
            color='white',
            fontsize=8,
            alpha=0.7,
            transform=fig.transFigure
        )
        
        # Add category counts as small colored indicators
        categories = {'person': 0, 'place': 0, 'animal': 0, 'generic': 0}
        for concept, data in self.concept_processor.concept_vectors.items():
            cat = data['category']
            categories[cat] = categories.get(cat, 0) + 1
            
        # Position for category indicators
        y_pos = 0.89
        x_start = 0.01
        width = 0.03
        height = 0.01
        spacing = 0.005
        
        # Add category count indicators
        for i, (cat, count) in enumerate(categories.items()):
            color = self.category_colors[cat]
            
            # Category color box
            cat_rect = plt.Rectangle(
                (x_start, y_pos - (i * (height + spacing))),
                width, height,
                transform=fig.transFigure,
                color=color,
                alpha=0.8
            )
            fig.patches.append(cat_rect)
            
            # Category count text
            fig.text(
                x_start + width + spacing, 
                y_pos - (i * (height + spacing)) + height/2,
                f"{cat.capitalize()}: {count}",
                color='white',
                fontsize=7,
                alpha=0.8,
                va='center',
                transform=fig.transFigure
            )
        
        return fig
        
    def visualize_concepts_advanced(self, output_dir, num_frames=60):
        """Create an advanced visualization with refined visual elements and better clarity"""
        try:
            print("Creating advanced semantic visualization...")
            
            if not self.concept_processor or not self.concept_processor.concept_vectors:
                raise ValueError("Concept processor not initialized or no vectors generated")
            
            # Get important relationships with higher threshold for better clarity
            try:
                important_relationships = self.concept_processor.compute_important_relationships(
                    threshold=0.6,  # Higher threshold for clearer relationships
                    max_relationships=40  # More relationships for richer visualization
                )
            except Exception as e:
                print(f"Warning: Could not compute relationships: {str(e)}")
                important_relationships = []
            
            # Setup figure with high DPI for crisp text
            plt.close('all')
            fig = plt.figure(figsize=(16, 12), dpi=180)  # Higher DPI for better quality
            ax = fig.add_subplot(111, projection='3d')
            
            # Apply the background gradient
            self._create_background_gradient(fig, ax)
            
            # Calculate node importance based on connectivity
            node_importance = {}
            for concept in self.concept_processor.concept_vectors.keys():
                # Count occurrences in important relationships
                count = sum(1 for rel in important_relationships 
                           if rel['source'] == concept or rel['target'] == concept)
                # Add a minimum size and scale importance more dramatically
                node_importance[concept] = 30 + (count * 25)  # Base size + importance factor
            
            def update(frame):
                try:
                    # Clear previous frame
                    ax.clear()
                    
                    # Handle text objects removal
                    for txt in fig.texts[:]:
                        fig.texts.remove(txt)
                    
                    # Calculate frame percentage for animations
                    frame_pct = frame / num_frames
                    
                    # Configure axis appearance
                    ax.grid(False)
                    ax.xaxis.pane.fill = False
                    ax.yaxis.pane.fill = False
                    ax.zaxis.pane.fill = False
                    
                    # Make panes completely transparent
                    ax.xaxis.pane.set_edgecolor('none')
                    ax.yaxis.pane.set_edgecolor('none')
                    ax.zaxis.pane.set_edgecolor('none')
                    
                    # Remove axis labels and ticks
                    ax.set_xticklabels([])
                    ax.set_yticklabels([])
                    ax.set_zticklabels([])
                    ax.set_xticks([])
                    ax.set_yticks([])
                    ax.set_zticks([])
                    
                    # Calculate dynamic camera angle with more complex movement
                    theta = frame_pct * 2 * np.pi
                    # Add a secondary oscillation for more interesting camera movement
                    phi = 0.2 * np.sin(frame_pct * 2 * np.pi * 0.5) + 0.3
                    # Add a slight zoom effect
                    zoom = 1 - 0.1 * np.sin(frame_pct * 2 * np.pi * 0.3)
                    
                    # Set viewing angle with enhanced transitions
                    ax.view_init(30 + 10 * np.sin(phi), 45 + 180 * frame_pct)
                    ax.dist = 8 * zoom  # Dynamic zoom level
                    
                    # Convert concept vectors to 3D points with dynamic positioning
                    points = {}
                    for concept, data in self.concept_processor.concept_vectors.items():
                        vector = data['vector'][:3]
                        
                        # Apply dynamic rotation matrix
                        x, y, z = vector
                        x_rot = x * np.cos(theta) - y * np.sin(theta)
                        y_rot = x * np.sin(theta) + y * np.cos(theta)
                        z_rot = z + 0.05 * np.sin(theta * 2 + x)  # Add gentle wave motion
                        
                        points[concept] = (x_rot, y_rot, z_rot)
                    
                    # Add depth cues first
                    self._add_depth_cues(ax, points)
                    
                    # Add enhanced relationship lines
                    self._enhance_relationship_lines(ax, points, important_relationships, frame_pct)
                    
                    # Plot points by category with enhanced visual elements
                    legend_elements = []
                    for category, color in self.category_colors.items():
                        cat_points = [
                            (concept, points[concept], node_importance[concept]) 
                            for concept in self.concept_processor.concept_vectors
                            if concept in points and 
                            self.concept_processor.concept_vectors[concept]['category'] == category
                        ]
                        
                        if cat_points:
                            # Split the data for plotting
                            concepts, coords, sizes = zip(*cat_points)
                            xs, ys, zs = zip(*coords)
                            
                            # Add improved glow effect
                            self._add_improved_glow_effect(ax, xs, ys, zs, color, sizes)
                            
                            # Create scatter plot with improved styling
                            scatter = ax.scatter(xs, ys, zs, 
                                              c=color, 
                                              alpha=0.9, 
                                              s=sizes,
                                              edgecolors='white',
                                              linewidth=0.5,
                                              zorder=10)
                            
                            # Add category to legend with improved styling
                            legend_elements.append(plt.Line2D(
                                [0], [0], 
                                marker='o', 
                                color='none',
                                markerfacecolor=color,
                                markeredgecolor='white',
                                markersize=10,
                                label=self.category_descriptions[category]))
                            
                            # Add labels for important concepts with improved styling
                            for concept, (x, y, z), size in zip(concepts, coords, sizes):
                                # Only label significant concepts for clarity
                                if len(concept) > 2 and size > 50:
                                    # Calculate label transparency based on size and position
                                    label_alpha = min(0.9, size / 100)
                                    
                                    # Create more visible text with better contrast
                                    ax.text(x, y, z + 0.05,
                                          concept,
                                          color='white',
                                          fontsize=8,
                                          fontweight='bold',
                                          alpha=label_alpha,
                                          backgroundcolor=(0, 0, 0, 0.5),
                                          ha='center',
                                          zorder=20)
                    
                    # Add interactive highlighting effect
                    self._add_interactive_highlighting(ax, points, frame_pct, node_importance)
                    
                    # Add visual cues to highlight key areas
                    self._add_visual_cues(ax, points, frame_pct)
                    
                    # Add legend with enhanced styling
                    legend = ax.legend(handles=legend_elements,
                                     loc='upper right',
                                     bbox_to_anchor=(0.99, 0.99),
                                     title='Semantic Categories',
                                     facecolor=(0.05, 0.05, 0.05, 0.8),
                                     edgecolor='white',
                                     fontsize=9)
                    
                    if legend:
                        legend.get_title().set_color('white')
                        for text in legend.get_texts():
                            text.set_color('white')
                    
                    # Add progress indicator
                    self._add_progress_indicator(fig, frame, num_frames)
                    
                    # Add advanced explanatory text and elements
                    self._add_advanced_explanatory_elements(fig, ax, frame_pct)
                    
                    return fig
                
                except Exception as e:
                    print(f"Error in update frame {frame}: {str(e)}")
                    traceback.print_exc()
                    return fig
            
            # Create animation with enhanced parameters
            anim = animation.FuncAnimation(
                fig, 
                update,
                frames=num_frames,
                interval=40,
                blit=False
            )
            
            # Create a static preview image
            print("Creating advanced static preview...")
            update(0)
            
            # Save with high quality settings
            static_path = os.path.join(output_dir, f'semantica_advanced_preview_{uuid.uuid4()}.png')
            plt.savefig(static_path, dpi=180, bbox_inches='tight')
            print(f"Preview image saved to: {static_path}")
            
            # Save animation with enhanced quality
            output_path = os.path.join(output_dir, f'semantica_advanced_{uuid.uuid4()}.gif')
            print(f"Generating animation (this may take some time)...")
            
            # Add a timestamp to the animation
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            fig.text(
                0.5, 0.01,
                f"Generated: {timestamp}",
                ha='center',
                color='white',
                fontsize=8,
                alpha=0.6,
                transform=fig.transFigure
            )
            
            # Use a more robust method to save the animation
            frames = []
            frame_files = []
            tmp_dir = os.path.join(output_dir, 'tmp_frames')
            
            # Create temporary directory for frames
            if not os.path.exists(tmp_dir):
                os.makedirs(tmp_dir)
                
            try:
                print("Generating frames...")
                for i in range(num_frames):
                    # Update the figure for this frame
                    update(i)
                    
                    # Save frame to a temporary file
                    tmp_file = os.path.join(tmp_dir, f'frame_{i:04d}.png')
                    plt.savefig(tmp_file, dpi=180, bbox_inches='tight',
                               facecolor=self.bg_gradient['bottom'])
                    frame_files.append(tmp_file)
                    
                    # Print progress
                    if i % 10 == 0 or i == num_frames - 1:
                        print(f"Generated frame {i+1}/{num_frames}")
                
                # Create GIF using PIL
                print("Saving animation...")
                from PIL import Image
                
                # Load all frames from files
                frames = [Image.open(f) for f in frame_files]
                
                # Ensure all frames are the same size
                if frames:
                    size = frames[0].size
                    frames = [f.resize(size) for f in frames]
                    
                    # Save as GIF
                    frames[0].save(
                        output_path,
                        save_all=True,
                        append_images=frames[1:],
                        optimize=False,
                        duration=1000/30,
                        loop=0
                    )
                    
                    # Close all frames
                    for f in frames:
                        f.close()
                    
                    print(f"Advanced visualization saved successfully to {output_path}")
            finally:
                # Clean up temporary files
                for file in frame_files:
                    try:
                        if os.path.exists(file):
                            os.remove(file)
                    except Exception as e:
                        print(f"Warning: Could not remove temporary file {file}: {e}")
                        
                # Try to remove temp directory
                try:
                    if os.path.exists(tmp_dir):
                        os.rmdir(tmp_dir)
                except Exception as e:
                    print(f"Warning: Could not remove temporary directory: {e}")
            
            plt.close()
            return output_path
            
        except Exception as e:
            print(f"Error creating visualization: {str(e)}")
            traceback.print_exc()
            return None

In [39]:
# Phase 3: Generate the advanced visualization
print("Starting Phase 3: Advanced visualization generation...")

# Initialize processor with the datasets we loaded at the beginning
print("Initializing ConceptNetProcessor with loaded datasets...")
processor = ConceptNetProcessor(english_conceptnet, german_conceptnet)

# Build semantic graph with reasonable sampling
print("Building semantic graph...")
processor.build_semantic_graph(
    max_concepts=10_000,  # Keep reasonable number of concepts for visualization
    min_weight=1.0,
    sample_size=0.5  # Use smaller sample for faster processing
)

# Check if the graph has nodes before generating vectors
if processor.semantic_graph.number_of_nodes() == 0:
    print("Warning: Semantic graph is empty. Creating a sample graph for visualization...")
    # Create a sample graph for demonstration purposes
    sample_concepts = {
        'person': ['human', 'woman', 'man', 'child', 'friend', 'family'],
        'place': ['home', 'city', 'park', 'office', 'school', 'mountain'],
        'animal': ['dog', 'cat', 'bird', 'fish', 'lion', 'tiger'],
        'generic': ['object', 'idea', 'concept', 'thought', 'action', 'time']
    }
    
    # Add nodes to graph
    for category, concepts in sample_concepts.items():
        for concept in concepts:
            processor.semantic_graph.add_node(concept, category=category)
            
    # Add some edges
    for category, concepts in sample_concepts.items():
        for i in range(len(concepts)-1):
            processor.semantic_graph.add_edge(concepts[i], concepts[i+1], relation='RelatedTo', weight=1.0)
    
    # Add cross-category relationships
    cross_relations = [
        (sample_concepts['person'][0], sample_concepts['place'][0]),
        (sample_concepts['person'][1], sample_concepts['animal'][0]),
        (sample_concepts['place'][2], sample_concepts['animal'][2]),
        (sample_concepts['generic'][0], sample_concepts['person'][3]),
        (sample_concepts['generic'][1], sample_concepts['place'][3]),
        (sample_concepts['generic'][2], sample_concepts['animal'][3])
    ]
    
    for source, target in cross_relations:
        processor.semantic_graph.add_edge(source, target, relation='RelatedTo', weight=1.0)
    
    print(f"Created sample graph with {processor.semantic_graph.number_of_nodes()} nodes and {processor.semantic_graph.number_of_edges()} edges")

# Generate concept vectors
print("Generating concept vectors...")
processor.generate_concept_vectors(dimensions=5)

# Create advanced visualizer with the processor
advanced_visualizer = AdvancedSemanticVisualizer(processor)

# Create the advanced visualization with improved depth cues
# and enhanced interactive elements
output_path = advanced_visualizer.visualize_concepts_advanced(
    output_dir,
    num_frames=300  # Reduced frames for faster rendering while still showing the features
)

print(f"Advanced visualization created at: {output_path}")
print("\nKey improvements in Phase 3:")
print("1. Enhanced depth perception with subtle grid reference")
print("2. Improved text legibility with better contrast backgrounds")
print("3. Interactive concept highlighting that changes focus over time")
print("4. Relationship strength indicators on important connections")
print("5. More informative category statistics and counts")
print("6. Dynamic camera movements with smooth zoom effects")
print("7. Enhanced visual prioritization of important concepts")
print("8. Higher resolution output for better detail")

Starting Phase 3: Advanced visualization generation...
Initializing ConceptNetProcessor with loaded datasets...
ConceptNetProcessor initialized
Building semantic graph...
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 1711502 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 539473 German assertions
Sampling 1711502 assertions from 3423004 en assertions
Sampling 1711502 assertions from 3423004 en assertions


Starting Phase 3: Advanced visualization generation...
Initializing ConceptNetProcessor with loaded datasets...
ConceptNetProcessor initialized
Building semantic graph...
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 1711502 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 539473 German assertions
Sampling 1711502 assertions from 3423004 en assertions
Sampling 1711502 assertions from 3423004 en assertions


Processing en assertions: 100%|██████████| 1711502/1711502 [00:42<00:00, 40174.13it/s]



Starting Phase 3: Advanced visualization generation...
Initializing ConceptNetProcessor with loaded datasets...
ConceptNetProcessor initialized
Building semantic graph...
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 1711502 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 539473 German assertions
Sampling 1711502 assertions from 3423004 en assertions
Sampling 1711502 assertions from 3423004 en assertions


Processing en assertions: 100%|██████████| 1711502/1711502 [00:42<00:00, 40174.13it/s]



...

Sampling 539473 assertions from 1078946 de assertions


Processing de assertions: 100%|██████████| 539473/539473 [00:14<00:00, 38478.26it/s]



Limiting graph to top 10000 concepts...
Semantic graph built with 10000 nodes and 65544 edges
Inferring semantic categories...
Inferred categories: {'generic': 8088, 'person': 470, 'place': 1442}
Generating concept vectors...
Generating 5-dimensional concept vectors...
Semantic graph built with 10000 nodes and 65544 edges
Inferring semantic categories...
Inferred categories: {'generic': 8088, 'person': 470, 'place': 1442}
Generating concept vectors...
Generating 5-dimensional concept vectors...
Generated vectors for 10000 concepts
Creating advanced semantic visualization...
Generated vectors for 10000 concepts
Creating advanced semantic visualization...
Creating advanced static preview...
Creating advanced static preview...


C:\Users\Erich Curtis\AppData\Local\Temp\ipykernel_61348\2616853387.py:313: MatplotlibDeprecationWarning: The dist attribute was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax.dist = 8 * zoom  # Dynamic zoom level


Preview image saved to: ../Data/Output\semantica_advanced_preview_69816660-0a18-486b-b517-59b5c21c84be.png
Generating animation (this may take some time)...
Generating frames...
Generated frame 1/300
Generated frame 1/300
Generated frame 11/300
Generated frame 11/300
Generated frame 21/300
Generated frame 21/300
Generated frame 31/300
Generated frame 31/300
Generated frame 41/300
Generated frame 41/300
Generated frame 51/300
Generated frame 51/300
Generated frame 61/300
Generated frame 61/300
Generated frame 71/300
Generated frame 71/300
Generated frame 81/300
Generated frame 81/300
Generated frame 91/300
Generated frame 91/300
Generated frame 101/300
Generated frame 101/300
Generated frame 111/300
Generated frame 111/300
Generated frame 121/300
Generated frame 121/300
Generated frame 131/300
Generated frame 131/300
Generated frame 141/300
Generated frame 141/300
Generated frame 151/300
Generated frame 151/300
Generated frame 161/300
Generated frame 161/300
Generated frame 171/300
Gene

# Phase 4